In [1]:
# 导入程序所需要的程序包

#抓取网页内容用的程序包
import json
import requests

#PyTorch用的包
import torch
import torch.nn as nn
import torch.optim
#from torch.autograd import Variable

# 自然语言处理相关的包
import re #正则表达式的包
import jieba #结巴分词包
from collections import Counter #搜集器，可以让统计词频更简单

#绘图、计算用的程序包
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import pandas as pd

from tqdm import tqdm,tqdm_notebook

In [2]:
content = pd.read_excel(r"./附件1/酒店评论.xlsx",engine='openpyxl')    
content_score = pd.read_excel(r"./附件2/酒店评分.xlsx",engine='openpyxl')    

In [3]:
content.head()

,酒店名称,评论日期,评论内容,入住房型
0,H01,2020-01-01,酒店很适合家庭出行,标准客房
1,H01,2020-01-01,升级了房间 延迟退房 很赞,标准客房
2,H01,2020-01-01,这几年，每年都会来广州，每次都会住**酒店。因为位置好，酒店的性价比也不错，这次给免费升级了...,标准客房
3,H01,2020-01-01,酒店很好不错,标准客房
4,H01,2020-01-01,超五星好评,高级客房


In [4]:
content_score.head()

,序号,酒店名称,总得分,服务得分,位置得分,设施得分,卫生得分,性价比得分
0,1,H01,4.8,4.8,4.8,4.7,4.8,4.0
1,2,H02,4.7,4.7,4.8,4.6,4.7,4.0
2,3,H03,4.8,4.8,4.8,4.8,4.9,4.0
3,4,H04,4.9,4.9,4.9,4.9,4.9,4.0
4,5,H05,4.7,4.7,4.8,4.7,4.8,4.0


In [5]:
data_merge = pd.merge(content,content_score)
data_merge.head()

,酒店名称,评论日期,评论内容,入住房型,序号,总得分,服务得分,位置得分,设施得分,卫生得分,性价比得分
0,H01,2020-01-01,酒店很适合家庭出行,标准客房,1,4.8,4.8,4.8,4.7,4.8,4.0
1,H01,2020-01-01,升级了房间 延迟退房 很赞,标准客房,1,4.8,4.8,4.8,4.7,4.8,4.0
2,H01,2020-01-01,这几年，每年都会来广州，每次都会住**酒店。因为位置好，酒店的性价比也不错，这次给免费升级了...,标准客房,1,4.8,4.8,4.8,4.7,4.8,4.0
3,H01,2020-01-01,酒店很好不错,标准客房,1,4.8,4.8,4.8,4.7,4.8,4.0
4,H01,2020-01-01,超五星好评,高级客房,1,4.8,4.8,4.8,4.7,4.8,4.0


In [6]:
# 将文本中的标点符号过滤掉
def filter_punc(sentence):
    sentence = re.sub("[\s+\.\!\/_,$%^*(+\"\'“”《》?“]+|[+——！，。？、~@#￥%……&*（）：]+|[0-9]", "", sentence)  
    return  sentence

In [7]:
#使用停用词
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

In [10]:
#扫描所有的文本，分词、建立词典，分出正向还是负向的评论，is_filter可以过滤是否筛选掉标点符号

all_words = [] #存储所有的单词
all_sentences = [] #存储正向的评论

#     data = content.loc[content['景区名称']==name]
stopwords = stopwordslist('stopwords.txt')

for idx,line in tqdm(enumerate(data_merge['评论内容'])):
    line = filter_punc(line)
    words = jieba.lcut(line)
    for i in words:
        if i in stopwords:
            words.remove(i)
#     if len(words) > 0:
    all_words = all_words+words
    all_sentences.append(words)

print('包含 {0} 行, {1} 个词.'.format( idx+1, len(all_words)))


25225it [01:44, 241.53it/s]

包含 25225 行, 309354 个词.


In [11]:
len(all_sentences)

25225

In [12]:
all_sentences[372]

[]

In [50]:
#建立词典，diction的每一项为{w:[id, 单词出现次数]}
diction = {}
cnt = Counter(all_words)
for word, freq in cnt.items():
    diction[word] = [len(diction), freq]

print('字典大小：{}'.format(len(diction)))

字典大小：21212


In [51]:
#根据单词返还单词的编码
def word2index(word, diction):
    if word in diction:
        value = diction[word][0]
    else:
        value = -1
    return(value)

#根据编码获得单词
def index2word(index, diction):
    for w,v in diction.items():
        if v[0] == index:
            return(w)
    return(None)

In [52]:
# 输入一个句子和相应的词典，得到这个句子的向量化表示
# 向量的尺寸为词典中词汇的个数，i位置上面的数值为第i个单词在sentence中出现的频率
def sentence2vec(sentence, dictionary):
    vector = np.zeros(len(dictionary))
    for l in sentence:
        vector[l] += 1
    return(1.0 * vector / len(sentence))

In [53]:

# 遍历所有句子，将每一个词映射成编码
dataset = [] #数据集
labels = data_merge['总得分'].values.tolist() #标签
sentences = [] #原始句子，调试用
# 处理正向评论
for sentence in all_sentences:
    new_sentence = []

    for l in sentence:
        if l in diction:
            new_sentence.append(word2index(l, diction))
    if len(sentence)>0:
        dataset.append(sentence2vec(new_sentence, diction))

        sentences.append(sentence)




In [58]:
len(dataset)

25175

In [59]:
len(labels)

25175

In [54]:
#打乱所有的数据顺序，形成数据集
# indices为所有数据下标的一个全排列
indices = np.random.permutation(len(dataset))

#重新根据打乱的下标生成数据集dataset，标签集labels，以及对应的原始句子sentences
dataset = [dataset[i] for i in indices]
labels = [labels[i] for i in indices]
sentences = [sentences[i] for i in indices]

#对整个数据集进行划分，分为：训练集、校准集和测试集，其中校准和测试集合的长度都是整个数据集的10分之一
test_size = len(dataset) // 10
train_data = dataset[2 * test_size :]
train_label = labels[2 * test_size :]

valid_data = dataset[: test_size]
valid_label = labels[: test_size]

test_data = dataset[test_size : 2 * test_size]
test_label = labels[test_size : 2 * test_size]

In [55]:
# 一个简单的前馈神经网络，三层，第一层线性层，加一个非线性ReLU，第二层线性层，中间有10个隐含层神经元

# 输入维度为词典的大小：每一段评论的词袋模型
model = nn.Sequential(
    nn.Linear(len(diction), 10),
    nn.ReLU(),
    nn.Linear(10, 1),
#     nn.LogSoftmax(),
)

def rightness(predictions, labels):
    """计算预测错误率的函数，其中predictions是模型给出的一组预测结果，batch_size行num_classes列的矩阵，labels是数据之中的正确答案"""
#     pred = torch.max(predictions.data, 1)[1] # 对于任意一行（一个样本）的输出值的第1个维度，求最大，得到每一行的最大元素的下标
#     print(predictions)
#     print(np.round(predictions[0][0].data,1))
    a=0
    rights =  np.round(predictions[0][0].data,1).eq(labels)
    if rights:
        a+=1
#     rights = predictions.eq(labels.data.view_as(predictions)).sum() #将下标与labels中包含的类别进行比较，并累计得到比较正确的数量
    return (a, len(labels)) #返回正确的数量和这一次一共比较了多少元素

In [56]:
model = model.double()

In [29]:
for i, data in enumerate(zip(train_data, train_label)):
        x, y = data
        
        # 需要将输入的数据进行适当的变形，主要是要多出一个batch_size的维度，也即第一个为1的维度
        x = torch.tensor(x, requires_grad = True).view(1,-1)
        # x的尺寸：batch_size=1, len_dictionary
        # 标签也要加一层外衣以变成1*1的张量
        y = torch.tensor(np.array([y]))
        # y的尺寸：batch_size=1, 1
#         print(y)
        # 清空梯度
        optimizer.zero_grad()
#         # 模型预测
        predict = model(x)
        print(predict)
        

tensor([[4.8194]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8619]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7312]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8827]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8780]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8558]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8337]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7649]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7873]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7786]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7670]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7027]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7805]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7772]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7708]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7482]], dtype=

tensor([[4.8488]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8350]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7343]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7783]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7833]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8171]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8476]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8350]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8688]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6682]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.9749]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6944]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8141]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8105]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7781]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6015]], dtype=

tensor([[4.6796]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7114]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7824]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6118]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8748]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7413]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7250]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7828]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8004]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6762]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.9078]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.9637]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7789]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7838]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6343]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7801]], dtype=

tensor([[4.8197]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.9706]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6839]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8429]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8735]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8775]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8000]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7456]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6434]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7298]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8948]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8347]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8002]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.5432]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8350]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[5.0066]], dtype=

tensor([[4.9830]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8948]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7926]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8055]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7967]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7855]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8827]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7435]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[5.0078]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8465]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.9316]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8412]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7741]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7757]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7322]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7576]], dtype=

...

tensor([[5.0361]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8003]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8827]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7362]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7852]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7095]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6572]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7666]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.9584]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7918]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8864]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8209]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8769]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7001]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7202]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7762]], dtype=

tensor([[4.6501]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8829]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8413]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8082]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.9071]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7780]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8553]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7555]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.9654]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7814]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8669]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7216]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7133]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6422]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6279]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7715]], dtype=

tensor([[4.8101]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8334]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8728]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8235]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7929]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6484]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6352]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8252]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.9013]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6865]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[5.0809]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7708]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8862]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6453]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7474]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8471]], dtype=

tensor([[4.7218]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7249]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8581]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.5939]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.9341]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7013]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8355]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6554]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.5858]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.5910]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7748]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8871]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.9149]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.9757]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7391]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8741]], dtype=

tensor([[4.7528]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6518]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7603]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7972]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6532]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8350]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8835]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7826]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.9895]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.6917]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.7712]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8210]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8284]], dtype=torch.float64, grad_fn=<AddmmBackward>)
tensor([[4.8507]], dtype=torch.float64, grad_fn=<AddmmBackward>)


In [57]:
# 损失函数为交叉熵
cost = torch.nn.MSELoss()
# 优化算法为Adam，可以自动调节学习率
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
records = []

#循环10个Epoch
losses = []
for epoch in range(10):
    for i, data in enumerate(zip(train_data, train_label)):
        x, y = data
        
        # 需要将输入的数据进行适当的变形，主要是要多出一个batch_size的维度，也即第一个为1的维度
        x = torch.tensor(x, requires_grad = True).view(1,-1)
        # x的尺寸：batch_size=1, len_dictionary
        # 标签也要加一层外衣以变成1*1的张量
        y = torch.tensor(np.array([y]))
        # y的尺寸：batch_size=1, 1
        
        # 清空梯度
        optimizer.zero_grad()
        # 模型预测
        predict = model(x)
        # 计算损失函数
        loss = cost(predict, y)
        # 将损失函数数值加入到列表中
        losses.append(loss.data.numpy())
        # 开始进行梯度反传
        loss.backward()
        # 开始对参数进行一步优化
        optimizer.step()
        
        # 每隔3000步，跑一下校验数据集的数据，输出临时结果
        if i % 3000 == 0:
            val_losses = []
            rights = []
            # 在所有校验数据集上实验
            for j, val in enumerate(zip(valid_data, valid_label)):
                x, y = val
                x = torch.tensor(x, requires_grad = True).view(1,-1)
                y = torch.tensor(np.array([y]))
                predict = model(x)
                # 调用rightness函数计算准确度
                right = rightness(predict, y)
                rights.append(right)
                loss = cost(predict, y)
                val_losses.append(loss.data.numpy())
                
            # 将校验集合上面的平均准确度计算出来
            right_ratio = 1.0 * np.sum([i[0] for i in rights]) / np.sum([i[1] for i in rights])
            print('第{}轮，训练损失：{:.2f}, 校验损失：{:.2f}, 校验准确率: {:.2f}'.format(epoch, np.mean(losses),
                                                                        np.mean(val_losses), right_ratio))
            records.append([np.mean(losses), np.mean(val_losses), right_ratio])
            

第0轮，训练损失：21.60, 校验损失：19.92, 校验准确率: 0.00
第0轮，训练损失：0.30, 校验损失：0.03, 校验准确率: 0.25
第0轮，训练损失：0.17, 校验损失：0.03, 校验准确率: 0.25
第0轮，训练损失：0.12, 校验损失：0.04, 校验准确率: 0.17
第0轮，训练损失：0.10, 校验损失：0.02, 校验准确率: 0.25
第0轮，训练损失：0.08, 校验损失：0.02, 校验准确率: 0.31
第0轮，训练损失：0.07, 校验损失：0.02, 校验准确率: 0.28
第1轮，训练损失：0.07, 校验损失：0.02, 校验准确率: 0.38
第1轮，训练损失：0.06, 校验损失：0.01, 校验准确率: 0.43
第1轮，训练损失：0.05, 校验损失：0.01, 校验准确率: 0.39
第1轮，训练损失：0.05, 校验损失：0.01, 校验准确率: 0.42
第1轮，训练损失：0.05, 校验损失：0.01, 校验准确率: 0.38
第1轮，训练损失：0.04, 校验损失：0.02, 校验准确率: 0.37
第1轮，训练损失：0.04, 校验损失：0.01, 校验准确率: 0.42
第2轮，训练损失：0.04, 校验损失：0.01, 校验准确率: 0.43
第2轮，训练损失：0.04, 校验损失：0.01, 校验准确率: 0.42
第2轮，训练损失：0.04, 校验损失：0.01, 校验准确率: 0.41
第2轮，训练损失：0.03, 校验损失：0.01, 校验准确率: 0.43
第2轮，训练损失：0.03, 校验损失：0.01, 校验准确率: 0.40
第2轮，训练损失：0.03, 校验损失：0.02, 校验准确率: 0.35
第2轮，训练损失：0.03, 校验损失：0.01, 校验准确率: 0.44
第3轮，训练损失：0.03, 校验损失：0.01, 校验准确率: 0.44
第3轮，训练损失：0.03, 校验损失：0.01, 校验准确率: 0.41
第3轮，训练损失：0.03, 校验损失：0.01, 校验准确率: 0.39
第3轮，训练损失：0.03, 校验损失：0.01, 校验准确率: 0.43
第3轮，训练损失：0.03, 校验损失：0.01, 校验准确率: 0.41
第3轮，训练损失：0